# **0. 참고자료**
## **0-1. 도서** 
- 소문난 명강의 - 김상형의 SQL 정복

## **0-2. 논문, 학술지**

## **0-3. 웹 사이트**

## **0-4. 데이터셋 출처**


In [1]:
import json
import sys
import os

from easydict import EasyDict as edict
import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-1])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-3])
sys.path.append(MISC_PATH)

from misc.config import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

## 노트북 재시작할때마다 DB삭제용 플래그
not_first = True

In [3]:
## DB 서버에 연결
conn   = sql.connect(host     = host, user = user, port = port,
                     password = passwd)

cursor = conn.cursor()

In [4]:
def print_results(text: str, results: tuple):
    
    print(f'{text}')
    for result in results: print(result)
    print('\n')

In [5]:
if not_first: cursor.execute('drop database study')

query = 'show databases'
cursor.execute(query)
print_results('[before]', cursor.fetchall())

## 생성하고자 하는 DB가 없는 경우에만 생성
cursor.execute('create database if not exists study')
cursor.execute('create database if not exists dummy')

cursor.execute(query)
print_results('[middle]', cursor.fetchall())

## DB 제거
cursor.execute('drop database dummy')
cursor.execute(query)
print_results('[after]', cursor.fetchall())

[before]
('DoveNest',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


[middle]
('DoveNest',)
('dummy',)
('information_schema',)
('mysql',)
('performance_schema',)
('study',)
('sys',)


[after]
('DoveNest',)
('information_schema',)
('mysql',)
('performance_schema',)
('study',)
('sys',)




In [6]:
## 사용하고자 하는 DB 선택
cursor.execute('use study')

query = 'show tables'
cursor.execute(query)
print_results('[before]', cursor.fetchall())

## 마찬가지로 생성하고자 하는 이름의 테이블이 없을때만 테이블 생성
crud_query = '''create table if not exists tCity
                (idx INTEGER AUTO_INCREMENT PRIMARY KEY,
                 NAME VARCHAR(20) NOT NULL,
                 AREA INTEGER NOT NULL,
                 POPU INTEGER NOT NULL,
                 METRO VARCHAR(5) NOT NULL,
                 REGION VARCHAR(10) NOT NULL)
             '''

dummy_query = '''
                create table if not exists dummy (idx INTEGER)
              '''

cursor.execute(crud_query)
cursor.execute(dummy_query)
cursor.execute(query)
print_results('[middle]', cursor.fetchall())

## 테이블 제거
cursor.execute('drop table dummy')
cursor.execute(query)
print_results('[after]', cursor.fetchall())

[before]


[middle]
('dummy',)
('tCity',)


[after]
('tCity',)




In [7]:
## 데이터 추가하는 쿼리함수
def insert(columns: list, values: list, table_name: str = 'tCity'):
    
    columns = ', '.join(columns)
    v       = ', '.join(['%s' for _ in values])
    query   = f'insert into {table_name} ({columns}) values ({v})'
    
    cursor.execute(query, values)

## **1. 데이터 조회 (SELECT 문)**
~~~ SQL
    SELECT '필드명' FROM '테이블 이름' [WHERE 조건][ORDER BY 정렬기준];
~~~
- []안에 있는 옵션들은 생략가능하다.
- 필드명에 '*'을 입력하면 모든 필드의 데이터를 가져올 수 있다.
    - 모든 필드의 데이터를 가져올 수는 있지만, 출력 결과가 가변적이라  
      위험한 경우도 있다.  
   - 코드가 불명확해지고, 어플리케이션의 메모리가 낭비되어 지양하는 것이 좋다. 
          

In [8]:
## 데이터 조회하는 쿼리함수
def select(column:str =  '*', table_name   : str = 'tCity', 
           cond  :str = None, order_column : str = None, order = None):
    
    query = f'select {column} from {table_name} '
    
    ## 조회하는데 조건이 있을 경우에 where문 추가
    if cond != None: query += f'where {cond}'
    
    ## 조회하는데 정렬하고자 하는 컬럼이 있다면 order by문 추가
    if order_column != None: 
        order  = 'desc' if order != None else 'asc'
        query += f'order by {order_column} {order}'
    
    cursor.execute(query)

In [9]:
select()
print_results('[before]', cursor.fetchall())


column = ['NAME', 'AREA', 'POPU', 'METRO', 'REGION']
values = [
          ['부산',    765,    342,     'y',    '경상'],
          ['서울',    605,    974,     'y',    '경기'],
          ['순천',    910,     27,     'n',    '전라'],
          ['오산',     42,     21,     'n',    '경기'],
          ['전주',    205,     65,     'n',    '전라'],
          ['청주',    940,     83,     'n',    '충청'],
          ['춘천',   1116,     27,     'n',    '강원'],
          ['홍천',   1819,      7,     'n',    '강원'],
         ]

for value in values: insert(column, value)

select()
print_results('[after]', cursor.fetchall())

[before]


[after]
(1, '부산', 765, 342, 'y', '경상')
(2, '서울', 605, 974, 'y', '경기')
(3, '순천', 910, 27, 'n', '전라')
(4, '오산', 42, 21, 'n', '경기')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')




In [10]:
select('REGION, POPU', order_column = 'POPU', order = 'desc')
print_results('[order test]', cursor.fetchall())

[order test]
('경기', 974)
('경상', 342)
('충청', 83)
('전라', 65)
('전라', 27)
('강원', 27)
('경기', 21)
('강원', 7)




In [11]:
select(cond = 'METRO = "n"', order_column = 'POPU')
print_results('[condition test]', cursor.fetchall())

[condition test]
(8, '홍천', 1819, 7, 'n', '강원')
(4, '오산', 42, 21, 'n', '경기')
(3, '순천', 910, 27, 'n', '전라')
(7, '춘천', 1116, 27, 'n', '강원')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')




In [12]:
## 다른 예제를 위한 새로운 테이블 생성
query = '''
            create table if not exists tStaff
            (name   CHAR(15) PRIMARY KEY,
             depart CHAR(10) NOT NULL,
             gender CHAR(3)  NOT NULL,
             joindate DATE NOT NULL,
             grade CHAR(10) NOT NULL,
             salary INT NOT NULL,
             score DECIMAL(5, 2) NULL)
        '''

cursor.execute(query)

0

In [13]:
column = ['name', 'depart', 'gender', 'joindate', 'grade', 'salary', 'score']
values = [
            ('김유신' , '총무부','남','2000-2-3','이사',420,88.8),
            ('유관순' , '영업부','여','2009-3-1','과장',380, None),
            ('안중근' , '인사과','남','2012-5-5','대리',256,76.5),
            ('윤봉길' , '영업부','남','2015-8-15','과장',350,71.25),
            ('강감찬' , '영업부','남','2018-10-9','사원',320,56.0),
            ('정몽주' , '총무부','남','2010-9-16','대리',370,89.5),
            ('허난설헌', '인사과','여','2020-1-5','사원',285,44.5),
            ('신사임당', '영업부','여','2013-6-19','부장',400,92.0),
            ('성삼문' , '영업부','남','2014-6-8','대리',285,87.75),
            ('논개'   , '인사과','여','2010-9-16','대리',340,46.2),
            ('황진이'  , '인사과','여','2012-5-5','사원',275,52.5),
            ('이율곡'  , '총무부','남','2016-3-8','과장',385,65.4),
            ('이사부'  , '총무부','남','2000-2-3','대리',375,50),
            ('안창호'  , '영업부','남','2015-8-15','사원',370,74.2),
            ('을지문덕' , '영업부','남','2019-6-29','사원',330, None),
            ('정약용'  , '총무부','남','2020-3-14','과장',380,69.8),
            ('홍길동'  , '인사과','남','2019-8-8','차장',380,77.7),
            ('대조영'  , '총무부','남','2020-7-7','차장',290,49.9),
            ('장보고'  , '인사과','남','2005-4-1','부장',440,58.3),
            ('선덕여왕' , '인사과','여','2017-8-3','사원',315,45.1),
         ]


for value in values: insert(column, value, table_name = 'tStaff')

select(table_name = 'tStaff')
print_results('[tStaff table]', cursor.fetchall())

[tStaff table]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 380, None)
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부'

##### **연습 문제 #001.**

> Q. tStaff 테이블에서 이름과 부서, 직급 필드만 출력하라


In [14]:
select('name, depart, grade', table_name = 'tStaff')
print_results('[A.]', cursor.fetchall())

[A.]
('강감찬', '영업부', '사원')
('김유신', '총무부', '이사')
('논개', '인사과', '대리')
('대조영', '총무부', '차장')
('선덕여왕', '인사과', '사원')
('성삼문', '영업부', '대리')
('신사임당', '영업부', '부장')
('안중근', '인사과', '대리')
('안창호', '영업부', '사원')
('유관순', '영업부', '과장')
('윤봉길', '영업부', '과장')
('을지문덕', '영업부', '사원')
('이사부', '총무부', '대리')
('이율곡', '총무부', '과장')
('장보고', '인사과', '부장')
('정몽주', '총무부', '대리')
('정약용', '총무부', '과장')
('허난설헌', '인사과', '사원')
('홍길동', '인사과', '차장')
('황진이', '인사과', '사원')




- SELECT 문이 출력하는 결과를 결과 셋(result set) 또는  
  로우 셋(row set) 이라고 하는 테이블이 생성된다.
  - 결과 셋의 필드명은 테이블에서 정의한 필드명과 동일하다.  
  
 ### **별명 (AS)**
 
- SELECT 문의 필드명 뒤에 AS를 사용해 별명을 설정하면,  
  출력 결과에서 설정한 별명으로 확인할 수 있다.
  - 공백이나 특수문자 포함이 가능하다.

In [15]:
select(column = 'name as 도시명, popu as "인구(만 명)"')

![AS 예제](../../../assets/SQL/CH4/AS_EXAMPLE.jpeg)

##### **연습 문제 #002.** 

> Q. 결과창의 헤더에 다음과 같이 출력하도록 하는 필드 목록을 작성하라.


||도시|인구(만명)|지역|
|---|---|---|---|
|1|서울|947|경기|
|2|부산|342|경상|
|3|오산|21|경기|
|4|청주|83|충청|

![연습문제 2번](../../../assets/SQL/CH4/EXER_#002.jpeg)

#### 계산값 출력
- SELECT 이후 필드명에 계산식을 이용하면 테이블에 저장된 값을 가공할 수 있다.
![계산식 적용](../../../assets/SQL/CH4/Calc.jpeg)


- 계산식을 이용하면 기존 테이블에 존재하지 않는 정보도 추가 할 수 있다.
![계산식 적용2](../../../assets/SQL/CH4/Calc2.jpeg)

- SELECT 문을 아래 둘 중 하나의 방법으로 간단한 계산기로도 사용할 수 있다.
    ~~~ SQL
        - select [계산식] from dual;  
        - select [계산식];
    ~~~

In [16]:
cursor.execute('select 60 * 60 * 24 as day from dual')
print_results('[with from]', cursor.fetchall())
cursor.execute('select 60 * 60 * 24 as day')
print_results('[without from]', cursor.fetchall())

[with from]
(86400,)


[without from]
(86400,)




##### **연습 문제 #003.** 

> Q. SELECT 문을 사용해 1년은 몇 초인지 계산하라.


In [17]:
cursor.execute('select 60 * 60 * 24 *365')
print_results('[A.]', cursor.fetchall())

[A.]
(31536000,)




## **2. 조건문**
### 2-1. 필드 비교
- WHERE 절은 읽을 레코드의 조건을 지정해준다.
    - SELECT는 조회가 주 기능이어서 주로 WHERE절과 사용한다.
    - WHERE 절은 DELETE, UPDATE 등의 명령과도 함께 사용된다.
    
- WHERE 절에서 조건에 사용할 수 있는 연산자는 아래와 같다.
    - 필드의 자료형과 비교대상의 자료형이 호환되어야 한다.  
    
- 숫자는 상수를 그대로 쓰지만, 문자열과 날짜 상수는 따옴표로 감싸야한다.
    
||연산자|설명|예|
|---|---|---|---|
|1|A = B|A와 B가 같다.|WHERE name='서울'|
|2|A <> B, A !=B|A와 B가 같지 않다.|WHERE region <> '경기'|
|3|A > B|A가 B보다 크다.|WHERE popu > 300|
|4|A < B|A가 B보다 작다.|WHERE popu < 200|
|5|A >= B|A가 B보다 크거나 같다.|WHERE area >= 1000|
|6|A <= B|A가 B보다 작거나 같다.|WHERE area <= 100|


In [18]:
select(cond = 'name = "서울"')
print_results('[name equal to "서울"]', cursor.fetchall())

select(cond = 'region <> "경기"')
print_results('[region not equal to "경기"]', cursor.fetchall())

select(cond = 'popu > 300')
print_results('[popu greater than 300]', cursor.fetchall())

select(cond = 'popu < 200')
print_results('[popu less than 200]', cursor.fetchall())

select(cond = 'area >= 1000')
print_results('[area greater than or equal to 3000]', cursor.fetchall())

select(cond = 'area <= 100')
print_results('[area less than or equal to 100]', cursor.fetchall())

[name equal to "서울"]
(2, '서울', 605, 974, 'y', '경기')


[region not equal to "경기"]
(1, '부산', 765, 342, 'y', '경상')
(3, '순천', 910, 27, 'n', '전라')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')


[popu greater than 300]
(1, '부산', 765, 342, 'y', '경상')
(2, '서울', 605, 974, 'y', '경기')


[popu less than 200]
(3, '순천', 910, 27, 'n', '전라')
(4, '오산', 42, 21, 'n', '경기')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')


[area greater than or equal to 3000]
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')


[area less than or equal to 100]
(4, '오산', 42, 21, 'n', '경기')




##### **연습 문제 #004.** 

> Q1. 인구가 10만명 미만인 도시의 이름을 출력하라.  
> Q2. 전라도에 있는 도시의 정보를 출력하라.  
> Q3. 월급이 400만원 이상인 직원의 이름을 출력하라.


In [19]:
select(column = 'name', cond = 'popu * 10000 > 100000')
print_results('[A1.]', cursor.fetchall())

select(cond = 'region = "전라"')
print_results('[A2.]', cursor.fetchall())

select(column = 'name', table_name = 'tStaff', cond = 'salary * 10000 >= 4000000')
print_results('[A3.]', cursor.fetchall())

[A1.]
('부산',)
('서울',)
('순천',)
('오산',)
('전주',)
('청주',)
('춘천',)


[A2.]
(3, '순천', 910, 27, 'n', '전라')
(5, '전주', 205, 65, 'n', '전라')


[A3.]
('김유신',)
('신사임당',)
('장보고',)




### 2-2. NULL 비교
- NULL은 값이 입력되어 있지 않은 상태를 말한다.
    - 값이 지정되어 있지 않기 때문에, 0이나 빈 문자열과도 다른 상태이다.
    
> #### <e.g.>
~~~ SQL  
    CREATE TABLE tCity
    (
        name CHAR(10) PRIMARY KEY,
        area INT NULL,
        popu INT NULL,
        metro CHAR(1) NOT NULL,
        region CHAR(6) NOT NULL
    );
~~~

- 선언문 뒤에 NULL이 있으면 값을 입력하지 않아도 된다는 뜻이다.
    - 위 예에서는 인구와 면적은 값을 입력하지 않아도 되고, 그 외 데이터 들은 입력해야한다.
    - NULL 비교를 위해서는 연산자로 is를 사용한다.

In [20]:
## tStaff 테이블에서 score가 NULL인 직원들만 추출
select(table_name = 'tStaff', cond = f'score is NULL')
print_results('[score is NULL in tStaff]', cursor.fetchall())

[score is NULL in tStaff]
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 380, None)
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)




In [21]:
select(table_name = 'tStaff', cond = f'score is not NULL')
print_results('[score is not NULL in tStaff]', cursor.fetchall())

[score is not NULL in tStaff]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40'))
('장보고', '인사과', '남', datetime.date(2005, 4, 1), '부장',

### 2-3. 논리 연산자
- 두 가지 이상의 조건을 동시에 점검할 때는 AND, OR 논리 연산자를 사용한다.
    - A AND B | A와 B 두 조건을 모두 만족하는 데이터
    - A OR  B | A와 B 둘 중 하나의 조건만 만족하는 데이터
    - AND 연산자가 OR 연산자보다 우선 순위가 높고, 괄호로 우선 순위를 지정해 줄 수 있다.
    
- 조건에 부합하지 않는 데이터를 구하기 위해서는 NOT 논리 연산자를 사용한다.
    - "!="와 같은 기능을 하지만, 복합 조건의 부정을 취할 때는 NOT을 사용하면 편하다.

In [22]:
select(cond = 'region = "경기" and popu >= 50 or area >= 500')
print_results('[지역이 경기도면서 인구가 50만 이상이거나 지역이 경기도가 아니여도 면적이 500 이상]', cursor.fetchall())

select(cond = 'region = "경기" and (popu >= 50 or area >= 500)')
print_results('[지역이 경기도면서, 인구가 50만이 넘거나 면적이 500 이상]', cursor.fetchall())

select(cond = 'region != "경기"')
print_results('[지역이 경기도가 아닌 데이터]', cursor.fetchall())

select(cond = 'NOT (region = "경기")')
print_results('[지역이 경기도가 아닌 데이터]', cursor.fetchall())

[지역이 경기도면서 인구가 50만 이상이거나 지역이 경기도가 아니여도 면적이 500 이상]
(1, '부산', 765, 342, 'y', '경상')
(2, '서울', 605, 974, 'y', '경기')
(3, '순천', 910, 27, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')


[지역이 경기도면서, 인구가 50만이 넘거나 면적이 500 이상]
(2, '서울', 605, 974, 'y', '경기')


[지역이 경기도가 아닌 데이터]
(1, '부산', 765, 342, 'y', '경상')
(3, '순천', 910, 27, 'n', '전라')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')


[지역이 경기도가 아닌 데이터]
(1, '부산', 765, 342, 'y', '경상')
(3, '순천', 910, 27, 'n', '전라')
(5, '전주', 205, 65, 'n', '전라')
(6, '청주', 940, 83, 'n', '충청')
(7, '춘천', 1116, 27, 'n', '강원')
(8, '홍천', 1819, 7, 'n', '강원')




##### **연습 문제 #005.** 

> Q1. 직원 목록에서 월급이 300 미만이면서 성취도는 60 이상인 직원이 누구인지 조사하라.  
> Q2. 영업부의 여직원 이름을 조사하라.


In [23]:
select(table_name = 'tStaff', cond = 'salary < 300 and score >= 60')
print_results('[A1.]', cursor.fetchall())

select(table_name = 'tStaff', column = 'name', cond = 'gender = "여"')
print_results('[A2.]', cursor.fetchall())

[A1.]
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))


[A2.]
('논개',)
('선덕여왕',)
('신사임당',)
('유관순',)
('허난설헌',)
('황진이',)


